# HuggingFace로 MNLI 자연어 추론 모델 구현하기

이번 실습에서는 HuggingFace를 사용하여 MNLI(Multi-Genre Natural Language Inference) 작업을 위한 모델을 구현합니다.
먼저 필요한 라이브러리들을 설치하고 임포트합니다.

In [27]:
!pip install transformers datasets evaluate accelerate scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Dataset 준비

MNLI 데이터셋을 `load_dataset` 함수로 다운로드 받습니다. MNLI는 두 문장(premise와 hypothesis)이 논리적으로 어떤 관계를 갖는지 분류하는 작업입니다.

In [29]:
mnli = load_dataset("nyu-mll/glue", "mnli")
mnli

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

`load_dataset`은 HuggingFace의 `datasets` 라이브러리의 함수로, HuggingFace의 hub에서 데이터셋을 다운로드 받을 수 있도록 만든 함수입니다.
MNLI 데이터셋은 `train`, `validation_matched`, `validation_mismatched`, `test_matched`, `test_mismatched` 등으로 구성되어 있습니다.
이번 과제에서는 지시에 따라 `train` 데이터만 사용하여 학습하고 검증합니다.

`train` 데이터를 한 번 살펴보겠습니다.

In [30]:
mnli['train'][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

데이터셋의 각 샘플에는 `premise`와 `hypothesis` 두 문장과 그들의 관계를 나타내는 `label`이 포함되어 있습니다.
- label 0: entailment (함의)
- label 1: neutral (중립)
- label 2: contradiction (모순)

이제 tokenizer를 불러와서 텍스트를 토큰화합니다. MNLI는 두 문장을 입력으로 받으므로 이를 처리할 수 있도록 전처리 함수를 작성합니다.

In [31]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(data["premise"], data["hypothesis"], truncation=True, padding="max_length")

mnli_tokenized = mnli.map(preprocess_function, batched=True)

/opt/homebrew/Caskroom/miniconda/base/envs/mcp/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [32]:
mnli_tokenized['train'][0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

이제 `train` 데이터를 분할하여 학습용과 검증용 데이터셋을 만듭니다.

In [33]:
mnli_split = mnli_tokenized['train'].train_test_split(test_size=0.1, seed=42)
mnli_train, mnli_val = mnli_split['train'], mnli_split['test']
mnli_validation = mnli_tokenized['validation_matched']

In [34]:
len(mnli_train), len(mnli_val), len(mnli_validation)

(353431, 39271, 9815)

## 모델 구현

이번에는 MNLI 작업을 위한 분류 모델을 구현합니다. HuggingFace의 사전훈련된 모델을 활용하여 세 가지 클래스(entailment, neutral, contradiction)를 예측하는 모델을 만듭니다.

In [35]:
id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", 
    num_labels=3, 
    id2label=id2label, 
    label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 학습 코드

모델을 MNLI 데이터셋으로 학습하기 위한 코드를 작성합니다.

In [36]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_mnli',  # 모델, 로그 등을 저장할 디렉토리
    num_train_epochs=3,  # 에폭 수
    per_device_train_batch_size=16,  # 학습 데이터의 배치 크기
    per_device_eval_batch_size=16,  # 검증 데이터의 배치 크기
    logging_strategy="epoch",  # 에폭이 끝날 때마다 로그
    do_train=True,  # 학습 진행
    do_eval=True,  # 검증 데이터에 대한 평가 수행
    eval_strategy="epoch",  # 매 에폭이 끝날 때마다 검증
    save_strategy="epoch",  # 매 에폭이 끝날 때마다 모델 저장
    learning_rate=2e-5,  # 학습률
    load_best_model_at_end=True,  # 학습이 끝난 후 가장 좋은 모델 선택
    metric_for_best_model="accuracy",  # 정확도를 기준으로 최고의 모델 선택
    evaluation_strategy="epoch"  # 매 에폭마다 평가
)

/opt/homebrew/Caskroom/miniconda/base/envs/mcp/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


각각의 부분들은 이전 주차에서 배웠던 내용들을 설정하는 것에 불과하다는 것을 알 수 있습니다.
요약하면 다음과 같습니다:
- `epochs`: training data를 몇 번 반복할 것인지 결정합니다.
- `batch_size`: training data를 얼마나 잘게 잘라서 학습할 것인지 결정합니다.
- `learning_rate`: optimizer의 learning rate를 얼마로 할 것인지 결정합니다.
위의 부분들 이외에도 사소한 구현 요소들도 지정할 수 있습니다.

다음은 loss 이외의 평가 함수들을 구현하는 방법입니다.

In [37]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

`evaluate` 또한 HuggingFace의 library로 다양한 평가 함수들을 제공하고 있습니다.
이번 실습의 경우, 감정 분석 문제는 분류 문제이기 때문에 정확도를 계산할 수 있습니다.
위와 같이 예측 결과(`pred`)와 실제 label(`labels`)가 주어졌을 때 정확도를 계산하는 것은 `evaluate`의 accuracy 함수로 구현할 수 있습니다.

마지막으로 위의 요소들을 종합하여 학습할 수 있는 `Trainer`를 구현합니다.

In [38]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mnli_train,
    eval_dataset=mnli_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

이제 모델을 학습합니다.

In [39]:
trainer.train()

  0%|          | 0/66270 [00:00<?, ?it/s]

KeyboardInterrupt: 

학습이 완료된 후, `validation_matched` 데이터셋에 대한 성능을 평가합니다. 목표는 50% 이상의 정확도를 달성하는 것입니다.

In [ ]:
results = trainer.evaluate(mnli_validation)
print(f"검증 데이터 정확도: {results['eval_accuracy']:.4f}")

  0%|          | 0/196 [00:00<?, ?it/s]

{'eval_loss': 0.3183744251728058,
 'eval_accuracy': 0.87064,
 'eval_runtime': 18.1086,
 'eval_samples_per_second': 1380.562,
 'eval_steps_per_second': 10.824,
 'epoch': 10.0}

이전에 학습 인자에서 `load_best_model_at_end=True`를 넘겨줬기 때문에 `trainer`는 학습이 끝난 후, 기본적으로 validation loss가 가장 좋은 모델을 가지고 `evaluate`를 진행합니다.
실제로 결과를 보면 `eval_loss`가 가장 낮은 validation loss와 유사한 것을 볼 수 있습니다.

평가할 때 사용한 모델은 다음과 같이 저장할 수 있습니다.

In [ ]:
trainer.save_model()

In [ ]:
classifier = pipeline("sentiment-analysis", model="./hf_transformer/")
print(classifier("The movie was so disgusting..."))
print(classifier("The movie was so amazing!!"))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEGATIVE', 'score': 0.9857644438743591}]
[{'label': 'POSITIVE', 'score': 0.998556911945343}]


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./hf_mnli/")

# 예시 1: entailment (함의) 예상
premise = "이 영화는 매우 흥미로웠습니다."
hypothesis = "저는 영화를 즐겼습니다."
print(classifier(premise + " " + hypothesis))

# 예시 2: contradiction (모순) 예상
premise = "이 책은 매우 어렵습니다."
hypothesis = "이 책은 아주 쉽습니다."
print(classifier(premise + " " + hypothesis))

# 예시 3: neutral (중립) 예상
premise = "그는 공원에서 달리기를 했습니다."
hypothesis = "그는 파란색 신발을 신었습니다."
print(classifier(premise + " " + hypothesis))